In [ ]:
import cv2
import os
import random
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [ ]:
# 젯슨 나노로 영상 촬영
video_output = 'captured_video.mp4'
cap = cv2.VideoCapture(0)

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output, fourcc, 20.0, (640, 640))

print('영상 촬영 시작 (종료: q 키)')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    out.write(frame)
    cv2.imshow('Recording...', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# 설정
video_path = 'video.mp4'  # 영상 파일 경로
output_dir = 'dataset'     # 저장 폴더
frame_interval = 96        # 프레임 간격
image_size = (640, 640)    # 이미지 크기

In [ ]:
# 폴더 생성
os.makedirs(f'{output_dir}/images/train', exist_ok=True)
os.makedirs(f'{output_dir}/images/val', exist_ok=True)
os.makedirs(f'{output_dir}/images/test', exist_ok=True)
os.makedirs(f'{output_dir}/labels/train', exist_ok=True)
os.makedirs(f'{output_dir}/labels/val', exist_ok=True)
os.makedirs(f'{output_dir}/labels/test', exist_ok=True)

In [ ]:
# 영상에서 프레임 추출
cap = cv2.VideoCapture(video_path)
frame_count = 0
image_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        frame = cv2.resize(frame, image_size)
        image_filename = f'frame_{image_count:04d}.jpg'
        image_path = os.path.join(output_dir, 'images', image_filename)
        cv2.imwrite(image_path, frame)

        # 바운딩 박스 자동 생성 (임계값 + 윤곽선 검출)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        label_filename = image_filename.replace('.jpg', '.txt')
        label_path = os.path.join(output_dir, 'labels', label_filename)

        with open(label_path, 'w') as f:
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)

                # 너무 작은 객체는 무시
                if w * h < 500:
                    continue

                # YOLO 형식으로 변환
                x_center = (x + w / 2) / image_size[0]
                y_center = (y + h / 2) / image_size[1]
                width = w / image_size[0]
                height = h / image_size[1]

                class_id = 0  # 단일 클래스라면 0으로 설정
                f.write(f'{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n')

        image_count += 1

    frame_count += 1

cap.release()

In [ ]:
# 데이터 분할
image_files = [f for f in os.listdir(f'{output_dir}/images') if f.endswith('.jpg')]
train_files, temp_files = train_test_split(image_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(temp_files, test_size=0.33, random_state=42)  # 약 7:2:1 비율

def move_files(files, subset):
    for file in files:
        img_src = os.path.join(output_dir, 'images', file)
        label_src = os.path.join(output_dir, 'labels', file.replace('.jpg', '.txt'))

        img_dst = os.path.join(output_dir, f'images/{subset}', file)
        label_dst = os.path.join(output_dir, f'labels/{subset}', file.replace('.jpg', '.txt'))

        os.rename(img_src, img_dst)
        os.rename(label_src, label_dst)

move_files(train_files, 'train')
move_files(val_files, 'val')
move_files(test_files, 'test')

print('데이터셋 준비 완료')

In [ ]:
# YOLOv8 모델 학습
model = YOLO('yolov8n.pt')  # YOLOv8n(경량화 모델) 사전 학습 모델 불러오기

# 학습 설정
model.train(
    data={
        'train': f'{output_dir}/images/train',
        'val': f'{output_dir}/images/val',
        'nc': 1,  # 클래스 수
        'names': ['object']  # 클래스 이름
    },
    epochs=50,        # 에폭 수
    batch=16,         # 배치 크기
    imgsz=640,        # 입력 이미지 크기
    project='yolo_project',  # 결과 저장 폴더
    name='exp1'       # 실험 이름
)

print('모델 학습 완료')

In [ ]:
# 실시간 트래킹 (젯슨 나노 웹캠)
model = YOLO('yolo_project/exp1/weights/best.pt')  # 학습된 모델 불러오기

# 젯슨 나노에서 기본 카메라 사용
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 수행
    results = model.track(frame, persist=True)  # 트래킹 유지

    # 탐지된 객체 시각화
    annotated_frame = results[0].plot()

    cv2.imshow('YOLOv8 Real-Time Tracking', annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()